In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from pandasql import sqldf
from sklearn.metrics import mean_squared_error

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_excel('C:\\Users\\osungar\\Desktop\projects\\sales_forecasting\\sales_project\\data\\satis_new.xlsx')


In [4]:
query = """
SELECT CARI_NO,STOK_NO,TOPLAM_BIRINCI_OB,OB1,TOPLAM_IKINCI_OB,OB2,SATIS_TARIHI                   
FROM df
"""

# Run the query
df_clean = sqldf(query, locals())
df_clean[:50]

,CARI_NO,STOK_NO,TOPLAM_BIRINCI_OB,OB1,TOPLAM_IKINCI_OB,OB2,SATIS_TARIHI
0,10636,1743,303.75,KG,7.000,AD,2015-04-01 00:00:00.000000
1,10657,743090,198.00,KG,1.564,B3,2015-04-01 00:00:00.000000
2,10636,1737,1558.50,KG,30.000,AD,2015-04-01 00:00:00.000000
3,10657,1979,268.00,KG,1.000,B3,2015-04-01 00:00:00.000000
4,10636,1736,762.00,KG,23.000,AD,2015-04-01 00:00:00.000000
5,10604,1988,132.00,KG,0.989,B3,2015-04-01 00:00:00.000000
6,10636,1725,1623.15,KG,32.000,AD,2015-04-01 00:00:00.000000
7,10636,1753,301.80,KG,9.000,AD,2015-04-01 00:00:00.000000
8,10604,1942,657.00,KG,4.954,B3,2015-04-01 00:00:00.000000
9,10636,1744,152.85,KG,3.000,AD,2015-04-01 00:00:00.000000


In [5]:
df_clean['SATIS_TARIHI'] = pd.to_datetime(df_clean['SATIS_TARIHI'])

In [6]:
filtered_data = df_clean[df_clean['SATIS_TARIHI'] < '2023-09-30']


In [7]:
filtered_data

,CARI_NO,STOK_NO,TOPLAM_BIRINCI_OB,OB1,TOPLAM_IKINCI_OB,OB2,SATIS_TARIHI
0,10636,1743,303.75,KG,7.000,AD,2015-04-01
1,10657,743090,198.00,KG,1.564,B3,2015-04-01
2,10636,1737,1558.50,KG,30.000,AD,2015-04-01
3,10657,1979,268.00,KG,1.000,B3,2015-04-01
4,10636,1736,762.00,KG,23.000,AD,2015-04-01
...,...,...,...,...,...,...,...
240729,10657,2004,545.00,KG,4.047,B5,2023-09-29
240730,55354,756398,48.00,AD,880.000,KG,2023-09-29
240731,55354,756715,96.00,AD,1267.550,KG,2023-09-29
240732,52917,747588,268.00,KG,2.006,B5,2023-09-29


In [8]:
df_date = filtered_data.copy()

In [9]:
df_date['YIL'] = df_clean['SATIS_TARIHI'].dt.year
df_date['AY'] = df_clean['SATIS_TARIHI'].dt.month

In [10]:
df_date = df_date.drop(columns=['SATIS_TARIHI'])

In [11]:
df_date_kg = df_date[df_date['OB1']=='KG']

In [12]:
df_date_kg = df_date_kg.drop(columns=['OB1'])

In [13]:
df_date_kg.tail()

,CARI_NO,STOK_NO,TOPLAM_BIRINCI_OB,TOPLAM_IKINCI_OB,OB2,YIL,AY
240725,10847,3142,121.40,5.000,AD,2023,9
240726,32944,751190,4739.00,22.578,B6,2023,9
240728,51758,751682,37.45,1.000,AD,2023,9
240729,10657,2004,545.00,4.047,B5,2023,9
240732,52917,747588,268.00,2.006,B5,2023,9


In [14]:
df_date_kg_ad = df_date_kg[df_date_kg['OB2']=='AD']

In [15]:
df_date_kg_ad = df_date_kg_ad.drop(columns=['OB2'])

In [16]:
query = """
SELECT CARI_NO,STOK_NO,AY,YIL,SUM(TOPLAM_BIRINCI_OB) as OB1_AYLIK_TOPLAM        
FROM df_date_kg_ad
group by CARI_NO,STOK_NO,YIL,AY
order by YIL,AY ASC
"""

# Run the query
df_month_kg_ad = sqldf(query, locals())
df_month_kg_ad.head(10)

,CARI_NO,STOK_NO,AY,YIL,OB1_AYLIK_TOPLAM
0,10443,4332,4,2015,870.85
1,10443,744535,4,2015,300.80
2,10563,3049,4,2015,61.20
3,10564,2594,4,2015,145.20
4,10564,2595,4,2015,103.90
5,10564,2602,4,2015,92.70
6,10564,2624,4,2015,98.80
7,10564,2626,4,2015,152.55
8,10564,2628,4,2015,218.85
9,10564,2663,4,2015,48.55


In [17]:
# Quarter sütunu oluştur
df_month_kg_ad['MEVSIM'] = pd.cut(df_month_kg_ad['AY'], bins=[0, 3, 6, 9, 12], labels=['q1', 'q2', 'q3', 'q4'])

# DataFrame'i görüntüle
df_month_kg_ad

,CARI_NO,STOK_NO,AY,YIL,OB1_AYLIK_TOPLAM,MEVSIM
0,10443,4332,4,2015,870.85,q2
1,10443,744535,4,2015,300.80,q2
2,10563,3049,4,2015,61.20,q2
3,10564,2594,4,2015,145.20,q2
4,10564,2595,4,2015,103.90,q2
...,...,...,...,...,...,...
70176,56187,754900,9,2023,469.40,q3
70177,56187,756157,9,2023,601.00,q3
70178,57210,2922,9,2023,228.75,q3
70179,57210,2924,9,2023,242.30,q3


In [18]:
categorical_columns=['CARI_NO', 'STOK_NO', 'AY', 'MEVSIM']
for col in categorical_columns:
    df_month_kg_ad[col] = df_month_kg_ad[col].astype('category')

In [19]:
#OB1_AYLIK_ORTALAMA is a target
X = df_month_kg_ad.drop(columns=['OB1_AYLIK_TOPLAM'])
y = df_month_kg_ad['OB1_AYLIK_TOPLAM']

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [21]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train,categorical_feature=categorical_columns)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [22]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'random_state':42
}

In [24]:
# Light Gradient Boosting Regressor
lgb_trained = lgb.train(params,
                lgb_train)


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001234 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1461
[LightGBM] [Info] Number of data points in the train set: 56144, number of used features: 5
[LightGBM] [Info] Start training from score 573.721319


In [25]:
y_pred =lgb_trained.predict(X_test)


In [26]:
print('MSE score on train data:')
print(mean_squared_error(y_test,y_pred))

MSE score on train data:
694482.3678053834


C:\anaconda3\envs\educaton\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\anaconda3\envs\educaton\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [27]:
merged_array = np.stack((y_test, y_pred), axis=1)


In [28]:
print("         y_test      ,      y_pred")
merged_array[:70]

         y_test      ,      y_pred


array([[3314.25      , 1419.8721428 ],
       [ 127.        ,  112.67534926],
       [ 168.85      ,  294.68090783],
       [  54.55      ,  -42.8816284 ],
       [  86.05      ,   98.44520248],
       [ 437.45      , 1191.22006085],
       [ 206.2       ,  125.07231468],
       [ 532.05      ,  268.75023108],
       [  80.9       ,  103.66208556],
       [  34.85      ,  239.53678861],
       [ 345.3       ,  147.38814503],
       [ 327.45      ,  179.48855762],
       [ 339.05      ,  167.28586269],
       [ 181.4       ,  143.00005883],
       [ 328.5       ,  250.88235249],
       [ 249.55      ,  230.0529944 ],
       [ 605.4       , 1395.10454335],
       [ 166.35      ,  504.58446775],
       [  65.2       ,  103.44646341],
       [  17.1       ,   45.9228632 ],
       [  36.55      ,  -30.75432076],
       [  19.3       ,   65.80915236],
       [ 224.3       ,  218.22205961],
       [2621.25      ,  907.11383727],
       [ 172.9       ,  433.0466854 ],
       [ 114.        ,   

# PREDICTING 10/2023


In [33]:
df_second = df_clean.copy()

In [34]:
df_second

,CARI_NO,STOK_NO,TOPLAM_BIRINCI_OB,OB1,TOPLAM_IKINCI_OB,OB2,SATIS_TARIHI
0,10636,1743,303.75,KG,7.000,AD,2015-04-01
1,10657,743090,198.00,KG,1.564,B3,2015-04-01
2,10636,1737,1558.50,KG,30.000,AD,2015-04-01
3,10657,1979,268.00,KG,1.000,B3,2015-04-01
4,10636,1736,762.00,KG,23.000,AD,2015-04-01
...,...,...,...,...,...,...,...
243846,55354,756313,7.00,AD,102.150,KG,2023-10-30
243847,10847,2897,51.25,KG,1.000,AD,2023-10-30
243848,10847,2966,249.85,KG,5.000,AD,2023-10-30
243849,10847,2980,115.95,KG,5.000,AD,2023-10-30


In [35]:
df_second = df_second[df_second["SATIS_TARIHI"] > '2023-09-30']
df_second = df_second[df_second["OB1"] == 'KG']
df_second = df_second[df_second["OB2"] == 'AD']

df_second['YIL'] = df_second['SATIS_TARIHI'].dt.year
df_second['AY'] = df_second['SATIS_TARIHI'].dt.month
df_second = df_second.drop(columns=['SATIS_TARIHI'])

query = """
SELECT CARI_NO,STOK_NO,AY,YIL,SUM(TOPLAM_BIRINCI_OB) as OB1_AYLIK_TOPLAM        
FROM df_second
group by CARI_NO,STOK_NO,YIL,AY
order by YIL,AY ASC
"""

# Run the query
df_second = sqldf(query, locals())
df_second

,CARI_NO,STOK_NO,AY,YIL,OB1_AYLIK_TOPLAM
0,10443,745358,10,2023,3121.75
1,10443,755789,10,2023,397.40
2,10564,2897,10,2023,99.05
3,10564,2899,10,2023,52.30
4,10564,2936,10,2023,693.35
...,...,...,...,...,...
727,57210,2922,10,2023,229.45
728,57210,2924,10,2023,485.45
729,57210,756157,10,2023,39.85
730,57439,3089,10,2023,170.95


In [36]:
df_second.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CARI_NO           732 non-null    int64  
 1   STOK_NO           732 non-null    int64  
 2   AY                732 non-null    int64  
 3   YIL               732 non-null    int64  
 4   OB1_AYLIK_TOPLAM  732 non-null    float64
dtypes: float64(1), int64(4)
memory usage: 28.7 KB


In [37]:
X

,CARI_NO,STOK_NO,AY,YIL,MEVSIM
0,10443,4332,4,2015,q2
1,10443,744535,4,2015,q2
2,10563,3049,4,2015,q2
3,10564,2594,4,2015,q2
4,10564,2595,4,2015,q2
...,...,...,...,...,...
70176,56187,754900,9,2023,q3
70177,56187,756157,9,2023,q3
70178,57210,2922,9,2023,q3
70179,57210,2924,9,2023,q3


In [38]:
df_prediction = X.copy()

In [39]:
df_prediction['AY'] = 10
df_prediction['YIL'] = 2023
df_prediction['MEVSIM'] = 'q4'


In [40]:
df_prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70181 entries, 0 to 70180
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   CARI_NO  70181 non-null  category
 1   STOK_NO  70181 non-null  category
 2   AY       70181 non-null  int64   
 3   YIL      70181 non-null  int64   
 4   MEVSIM   70181 non-null  object  
dtypes: category(2), int64(2), object(1)
memory usage: 2.0+ MB


In [41]:
categorical_columns=['CARI_NO', 'STOK_NO', 'AY','MEVSIM']
for col in categorical_columns:
    df_prediction[col] = df_prediction[col].astype('category')

In [42]:
df_prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70181 entries, 0 to 70180
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   CARI_NO  70181 non-null  category
 1   STOK_NO  70181 non-null  category
 2   AY       70181 non-null  category
 3   YIL      70181 non-null  int64   
 4   MEVSIM   70181 non-null  category
dtypes: category(4), int64(1)
memory usage: 1.0 MB


In [43]:
df_prediction = df_prediction.drop_duplicates(subset=['CARI_NO', 'STOK_NO'])
df_prediction

,CARI_NO,STOK_NO,AY,YIL,MEVSIM
0,10443,4332,10,2023,q4
1,10443,744535,10,2023,q4
2,10563,3049,10,2023,q4
3,10564,2594,10,2023,q4
4,10564,2595,10,2023,q4
...,...,...,...,...,...
70171,56187,753765,10,2023,q4
70172,56187,754513,10,2023,q4
70174,56187,754670,10,2023,q4
70178,57210,2922,10,2023,q4


In [44]:
df_prediction.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9538 entries, 0 to 70179
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   CARI_NO  9538 non-null   category
 1   STOK_NO  9538 non-null   category
 2   AY       9538 non-null   category
 3   YIL      9538 non-null   int64   
 4   MEVSIM   9538 non-null   category
dtypes: category(4), int64(1)
memory usage: 293.9 KB


In [45]:
y_pred =lgb_trained.predict(df_prediction)

In [46]:
df_prediction

,CARI_NO,STOK_NO,AY,YIL,MEVSIM
0,10443,4332,10,2023,q4
1,10443,744535,10,2023,q4
2,10563,3049,10,2023,q4
3,10564,2594,10,2023,q4
4,10564,2595,10,2023,q4
...,...,...,...,...,...
70171,56187,753765,10,2023,q4
70172,56187,754513,10,2023,q4
70174,56187,754670,10,2023,q4
70178,57210,2922,10,2023,q4


In [47]:
df_prediction['TAHMIN'] = y_pred

In [48]:
df_prediction

,CARI_NO,STOK_NO,AY,YIL,MEVSIM,TAHMIN
0,10443,4332,10,2023,q4,1102.640581
1,10443,744535,10,2023,q4,940.912152
2,10563,3049,10,2023,q4,614.971785
3,10564,2594,10,2023,q4,64.203752
4,10564,2595,10,2023,q4,143.754785
...,...,...,...,...,...,...
70171,56187,753765,10,2023,q4,313.020770
70172,56187,754513,10,2023,q4,346.960353
70174,56187,754670,10,2023,q4,430.115970
70178,57210,2922,10,2023,q4,940.401166


In [49]:
df_second

,CARI_NO,STOK_NO,AY,YIL,OB1_AYLIK_TOPLAM
0,10443,745358,10,2023,3121.75
1,10443,755789,10,2023,397.40
2,10564,2897,10,2023,99.05
3,10564,2899,10,2023,52.30
4,10564,2936,10,2023,693.35
...,...,...,...,...,...
727,57210,2922,10,2023,229.45
728,57210,2924,10,2023,485.45
729,57210,756157,10,2023,39.85
730,57439,3089,10,2023,170.95


In [50]:
query = """
SELECT p.CARI_NO,s.STOK_NO,p.AY,p.YIL,p.MEVSIM,p.TAHMIN,s.OB1_AYLIK_TOPLAM
FROM df_second as s
LEFT JOIN df_prediction as p ON s.CARI_NO = p.CARI_NO AND s.STOK_NO = p.STOK_NO;
"""

# Run the query
df_merged = sqldf(query, locals())
df_merged.head(50)

,CARI_NO,STOK_NO,AY,YIL,MEVSIM,TAHMIN,OB1_AYLIK_TOPLAM
0,10443,745358,10,2023.0,q4,2707.651790,3121.75
1,10443,755789,10,2023.0,q4,927.342149,397.40
2,10564,2897,10,2023.0,q4,320.507860,99.05
3,10564,2899,10,2023.0,q4,180.960442,52.30
4,10564,2936,10,2023.0,q4,458.573985,693.35
5,10564,2938,10,2023.0,q4,405.502022,458.10
6,10564,2964,10,2023.0,q4,99.174687,48.85
7,10564,2968,10,2023.0,q4,158.337679,104.20
8,10564,3065,10,2023.0,q4,93.610967,159.50
9,10564,3075,10,2023.0,q4,84.263093,142.50


In [51]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CARI_NO           677 non-null    object 
 1   STOK_NO           732 non-null    int64  
 2   AY                677 non-null    object 
 3   YIL               677 non-null    float64
 4   MEVSIM            677 non-null    object 
 5   TAHMIN            677 non-null    float64
 6   OB1_AYLIK_TOPLAM  732 non-null    float64
dtypes: float64(3), int64(1), object(3)
memory usage: 40.2+ KB


In [52]:
df_merged.isnull().sum()

CARI_NO             55
STOK_NO              0
AY                  55
YIL                 55
MEVSIM              55
TAHMIN              55
OB1_AYLIK_TOPLAM     0
dtype: int64

In [53]:
query = """
SELECT *
FROM df_merged
where CARI_NO IS NULL
"""

# Run the query
test = sqldf(query, locals())
test

,CARI_NO,STOK_NO,AY,YIL,MEVSIM,TAHMIN,OB1_AYLIK_TOPLAM
0,None,2437,None,None,None,None,35.90
1,None,2472,None,None,None,None,140.35
2,None,2474,None,None,None,None,311.05
3,None,2476,None,None,None,None,380.50
4,None,2561,None,None,None,None,14.40
5,None,2625,None,None,None,None,64.85
6,None,2935,None,None,None,None,691.25
7,None,3010,None,None,None,None,18.05
8,None,3012,None,None,None,None,391.85
9,None,4166,None,None,None,None,34.05


In [54]:
merged = df_merged.dropna()

In [56]:
query = """
SELECT *
FROM merged
order by OB1_AYLIK_TOPLAM DESC;
"""

# Run the query
df_last = sqldf(query, locals())
df_last.head(50)

,CARI_NO,STOK_NO,AY,YIL,MEVSIM,TAHMIN,OB1_AYLIK_TOPLAM
0,10599,757032,10,2023.0,q4,1253.292603,23690.50
1,10652,743069,10,2023.0,q4,10878.170886,10205.00
2,10700,757700,10,2023.0,q4,1916.222799,7100.70
3,55963,757622,10,2023.0,q4,433.557121,6274.25
4,42154,2924,10,2023.0,q4,3222.876641,5468.65
5,47698,751781,10,2023.0,q4,7114.017027,5372.35
6,10847,743463,10,2023.0,q4,2638.160866,5266.20
7,42154,2936,10,2023.0,q4,4560.570583,3909.70
8,42154,2934,10,2023.0,q4,3542.742724,3230.80
9,51970,2614,10,2023.0,q4,567.027280,3192.10


In [57]:
df_last2 = df_last.copy()

In [58]:
df_last2['TAHMIN'] = df_last2['TAHMIN']*1.1

In [59]:
df_last2[:50]

,CARI_NO,STOK_NO,AY,YIL,MEVSIM,TAHMIN,OB1_AYLIK_TOPLAM
0,10599,757032,10,2023.0,q4,1378.621863,23690.50
1,10652,743069,10,2023.0,q4,11965.987975,10205.00
2,10700,757700,10,2023.0,q4,2107.845079,7100.70
3,55963,757622,10,2023.0,q4,476.912833,6274.25
4,42154,2924,10,2023.0,q4,3545.164305,5468.65
5,47698,751781,10,2023.0,q4,7825.418730,5372.35
6,10847,743463,10,2023.0,q4,2901.976953,5266.20
7,42154,2936,10,2023.0,q4,5016.627642,3909.70
8,42154,2934,10,2023.0,q4,3897.016996,3230.80
9,51970,2614,10,2023.0,q4,623.730008,3192.10
